In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf

2022-01-11 12:44:30.389003: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Using TensorFlow backend.


In [2]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [4]:
print(tf.__version__)

2.4.1


In [5]:

tf.config.list_physical_devices('GPU')
#print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

2022-01-11 12:44:32.245536: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

2022-01-11 12:44:32.246758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-11 12:44:32.298087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 12:44:32.298438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 12:44:32.298468: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-11 12:44:32.300243: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-11 12:44:32.300279: I tensorflow/stream_executor/platfo

In [6]:
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPUs:  1


In [7]:
train = pd.read_csv('train_metadata.csv')
test = pd.read_csv('test_metadata.csv')
#submission = pd.read_csv('sample_submission.csv')

train_dir = 'train/'
test_dir = 'test/'

In [8]:
label = []
train_data = []
for i in range(train.shape[0]):
    train_data.append(train_dir + train['image_name'].iloc[i] + '.jpg')
    label.append(train['diagnosis'].iloc[i])
    
test_data = []
for i in range(test.shape[0]):
    test_data.append(test_dir + test['image'].iloc[i] + '.jpg')

In [9]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
label_df = pd.DataFrame(label, columns = ['diagnosis'])
one_hot_encoder.fit(label_df)

label_df_encoded = one_hot_encoder.transform(label_df)
label_df_encoded = pd.DataFrame(data=label_df_encoded, columns=one_hot_encoder.categories_)
print(label_df_encoded.columns)

MultiIndex([('atypical melanocytic proliferation',),
            (               'cafe-au-lait macule',),
            (                       'lentigo NOS',),
            (               'lichenoid keratosis',),
            (                          'melanoma',),
            (                             'nevus',),
            (              'seborrheic keratosis',),
            (                     'solar lentigo',),
            (                           'unknown',)],
           )


In [10]:
df_train = pd.DataFrame(train_data)
df_train.columns = ['images']
df_train = pd.concat([df_train, label_df_encoded], axis = 1)
print(df_train.columns)
df_test = pd.DataFrame(test_data)
df_test.columns = ['images']

Index([                               'images',
       ('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_train.iloc[:, 0], df_train.iloc[:,1::], test_size = 0.3, random_state = 42)

train = pd.DataFrame(X_train)
train.columns = ['images']
train = pd.concat([train, y_train], axis = 1)

test = pd.DataFrame(X_test)
test.columns = ['images']
test = pd.concat([test, y_test], axis = 1)

In [12]:
train.columns[1:]

Index([('atypical melanocytic proliferation',),
                      ('cafe-au-lait macule',),
                              ('lentigo NOS',),
                      ('lichenoid keratosis',),
                                 ('melanoma',),
                                    ('nevus',),
                     ('seborrheic keratosis',),
                            ('solar lentigo',),
                                  ('unknown',)],
      dtype='object')

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=50,
                                   zoom_range = 0.3,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train,
    x_col=train.columns[0],
    y_col=train.columns[1:],
    target_size=(224, 224),
    batch_size=8,
    shuffle=True,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    test,
    x_col=test.columns[0],
    y_col=test.columns[1:],
    target_size=(224, 224),
    shuffle=False,
    batch_size=8,
    class_mode='raw')

Found 23188 validated image filenames.
Found 9938 validated image filenames.


In [14]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
input_shape = (224,224,3)
vgg = vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_shape=input_shape)

output = vgg.layers[-1].output
output = keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

vgg_model.trainable = False
for layer in vgg_model.layers:
    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

2022-01-11 12:44:32.887187: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-11 12:44:32.887542: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-11 12:44:32.887678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-11 12:44:32.888018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-11 12:44

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f94d04236d0>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f94d0e048b0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f94d0e04a60>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f94d0e04d30>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7e3400>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7e3760>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f94d0e04e80>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7f0130>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7f0fa0>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7ebd30>,block3_conv3,False


In [15]:
vgg_model.trainable = True

set_trainable = False
for layer in vgg_model.layers:
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
layers = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f94d04236d0>,input_1,False
1,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f94d0e048b0>,block1_conv1,False
2,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f94d0e04a60>,block1_conv2,False
3,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f94d0e04d30>,block1_pool,False
4,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7e3400>,block2_conv1,False
5,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7e3760>,block2_conv2,False
6,<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f94d0e04e80>,block2_pool,False
7,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7f0130>,block3_conv1,False
8,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7f0fa0>,block3_conv2,False
9,<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f952a7ebd30>,block3_conv3,False


In [16]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
import tensorflow as tf

class_num = 9

model = Sequential()
model.add(vgg_model)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(class_num, activation='softmax'))

opt = optimizers.Adam(lr=0.0001)
model.compile(loss="categorical_crossentropy", metrics= 'accuracy',optimizer=opt)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 25088)             14714688  
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 4617      
Total params: 27,827,529
Trainable params: 26,092,041
Non-trainable params: 1,735,488
____________________________________

In [18]:
batch_size = 128
num_classes = 2
epochs = 100
nb_train_steps = train.shape[0] // batch_size
nb_val_steps = test.shape[0] // batch_size

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=5, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=50, verbose=1, mode='auto')

model.fit_generator(train_generator, steps_per_epoch=nb_train_steps, epochs=epochs,
                              validation_data=test_generator, validation_steps=nb_val_steps, 
                              verbose=1, callbacks=[checkpoint,early])

/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-11 12:44:34.233685: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-11 12:44:34.254215: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4200000000 Hz


Epoch 1/100


2022-01-11 12:44:34.719047: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-01-11 12:44:34.906397: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-01-11 12:44:35.036964: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


  1/181 [..............................] - ETA: 7:58 - loss: 2.1510 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [21]:
#Evaluate
model = keras.models.load_model("Models/vgg16_1.h5")
scores = model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

scores = model.evaluate_generator(test_generator)
print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))

1243/1243 [==============================] - 380s 305ms/step - loss: 0.2352 - accuracy: 0.9467
evaluate accuracy: 94.67%


/home/deeplab/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


evaluate_generator accuracy: 94.67%


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

data = test_datagen.flow_from_dataframe(dataframe = df_train.iloc[:100,:],
      x_col="images", class_mode=None, 
      batch_size=1, target_size=(224,224), shuffle=False)

predictions= model.predict_generator(data, max_queue_size=64, workers = 32, verbose=1)
print(predictions[:100,0])

In [ ]:
print(predictions)

In [ ]:
train.columns